In [2]:
import os

In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone Github repositories

In [4]:
%pwd

'd:\\Bappy\\Office\\Live-Session\\GenAI\\Source-Code-Analysis-using-GenAI\\research'

In [5]:
!mkdir test_repo

In [6]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation", to_path=repo_path)

<git.repo.base.Repo 'd:\\Bappy\\Office\\Live-Session\\GenAI\\Source-Code-Analysis-using-GenAI\\research\\test_repo\\.git'>